In [ ]:
import torch
from datasets import load_mnist, get_observation_pixels
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim

BATCH_SIZE = 100
train_loader, test_loader, val_loader = load_mnist(BATCH_SIZE)

In [3]:
import pytorch_lightning as pl

from models import VAE

class LinearVAETrainer(pl.LightningModule):
    def __init__(self, hidden_dims = [512, 256], latent_dim = 2):
        super(LinearVAETrainer, self).__init__()
        
        self.model = VAE(hidden_dims, latent_dim)
        
    def forward(self, x, x_cond):
        return self.model(x)
    
    def step(self, batch, batch_idx, mode = 'train'):
        x, x_cond, y = batch
        x_hat, mu, log_var = self(x, x_cond)
        loss = self.model.loss(x, x_hat, mu, log_var)
        self.log_dict({f"{mode}_{key}": val.item() for key, val in loss.items()}, sync_dist=True)
        return loss['loss']
    
    def training_step(self, batch, batch_idx):
        return self.step(batch, batch_idx, 'train')
    def validation_step(self, batch, batch_idx):
        return self.step(batch, batch_idx, 'val')
    def test_step(self, batch, batch_idx):
        return self.step(batch, batch_idx, 'test')
    
    def configure_optimizers(self):
        return optim.Adam(self.model.parameters(), lr=1e-3)

ImportError: cannot import name 'VAE' from 'models' (/Home/siv34/edzak2974/projects/MastersThesis/src/models/__init__.py)